In [1]:
import time
from datetime import datetime, timedelta
from dateutil.relativedelta import *
import pandas as pd
import os
import csv
from scipy.stats import pearsonr
import numpy as np
import re
import pytrends
import bash
import random
import pdb
import calendar
import requests
from requests.exceptions import Timeout
from pytrends.exceptions import ResponseError
from torpy.circuit import  CellTimeoutError

In [2]:
def get_cookie():
    from selenium import webdriver
    options = webdriver.EdgeOptions()
    edge_driver_path = r'C:\Users\Sean Liu\Downloads\edgedriver_win64\edge_webdriver.exe'

# Create a WebDriver instance for Microsoft Edge
    driver = webdriver.Edge(options = options, executable_path=edge_driver_path)   
    driver.get("https://trends.google.com/")
    time.sleep(5)
    cookie = driver.get_cookie("NID")["value"]
    driver.quit()
    return cookie

nid_cookie = f"NID={get_cookie()}"


C:\Users\Sean Liu\AppData\Local\Temp\ipykernel_236\3900106164.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Edge(options = options, executable_path=edge_driver_path)


In [3]:
nid_cookie

'NID=511=heyTdTvGUFolQPv8kAi2xgRSCeojbf_QJxDHWSfestUUgKJzCbQDhMfAfuP-Zo4-sRJD5XRFlS3oLtv_2lp5YCcq4jhVshHN7zB9wS8SVQGmUrW3-95CNNRa14wJbPLweXcaUnsRDWVx-phV830VnTDvrzVKBZgZRA2mjFDKkm4'

In [4]:
from pytrends.request import TrendReq

requests_args = {
    'headers' : {
        "Cookie": nid_cookie,
    }
}
 



In [5]:
def init():
    #https://pypi.org/project/torpy/
    #pip install torpy
    from torpy import TorClient
    hostname = 'ifconfig.me'  # It's possible use onion hostname here as well
    with TorClient() as tor:
        # Choose random guard node and create 3-hops circuit
        with tor.create_circuit(3) as circuit:
            # Create tor stream to host
            with circuit.create_stream((hostname, 80)) as stream:
                # Now we can communicate with host
                stream.send(b'GET / HTTP/1.0\r\nHost: %s\r\n\r\n' % hostname.encode())
                recv = stream.recv(1024)
    return 0

In [6]:
def column_name_to_mvt_and_remove_zero_values(df, geo_name):
    df.rename(columns={df.columns[0]: f"MVT_GT_{geo_name}" }, inplace = True)
    df = df[df[f"MVT_GT_{geo_name}"] != 0]
    return df

In [7]:
def execute_init():
    while True:
        try:
            init()
            print("init status completed")
            break
        except (TimeoutError, CellTimeoutError, ConnectionResetError) as error:
            print(f"Init Error Occured: {error}")
            print("Let me sleep for 10 seconds")
            print("ZZzzzz...")
            time.sleep(10)
            print("Was a nice sleep, now let me continue...")
            pass
    return 0

In [8]:
def GT_temp_df(start_date, end_date, geo_code, geo_name):
    
    temp_df = pd.DataFrame()
    while len(temp_df) == 0:
        try:
            pytrend.build_payload(mvt_keyword, cat=0, timeframe=f"{start_date} {end_date}", geo = geo_code)
            temp_df = pytrend.interest_over_time()

        except (ResponseError, Timeout, ConnectionAbortedError, ConnectionError) as error:
            try:
                print(f"GT Error Occured: {error}")
                !pip install pytrends --upgrade
                #init()
                #rint(f"Let's Go, Onion!")
                pytrend.build_payload(mvt_keyword, cat=0, timeframe=f"{start_date} {end_date}", geo = geo_code)
                temp_df = pytrend.interest_over_time()

            except (ResponseError, Timeout, ConnectionAbortedError, ConnectionError) as error:
                print(f"GT Error Occured: {error}")
                print(f"Let me sleep for 1 day, till {(datetime.now() + timedelta(days=1)).strftime('%m/%d/%Y, %H:%M:%S')}")
                print("ZZzzzz...")
                time.sleep(random.randint(60*60*24, 60*60*24+30,))
                print("Was a nice sleep, now let me continue...")


    print(f"Obtaining GT MVT from {start_date} to {end_date} in {geo_name} metro area")
    time.sleep(random.randint(600, 660))
    temp_df = column_name_to_mvt_and_remove_zero_values(temp_df, geo_name)
    return temp_df

In [9]:
def combine_df_and_temp_df_and_rescale_by_overlap_factors(df, temp_df):
    df = pd.concat([df, temp_df.iloc[:, 0]], axis = 1)
    if len(df.columns) > 1:
        factor_data = df.dropna(how = "any")
        factor_divided = factor_data.iloc[:, 0]/factor_data.iloc[:, 1]
        adjusted_factor = factor_divided.mean()
        df.iloc[:,1] = df.iloc[:,1]*adjusted_factor
        df = df.mean(axis = 1)
    return df

In [ ]:
init()

In [ ]:
#basic pytrend settings
pytrend = TrendReq(requests_args=requests_args,  retries=3)

In [ ]:
'''
kw_list = ["car stolen+find stolen car+report police stolen car+insurance car stolen-dream-check",
    "I raped+raped me+raped report police+being raped+been raped - dream - kobe - trump - porn",
    "my stolen+find my stolen+theft report police+someone stolen my - car - heart - dream - number",
    "burglary+home burglarized+my house broken into+my home broken into+burglary report police - dream",
    "my committed a crime+police arrested my+my arrested+my in jail+my in prison - arrested development",
    "I hate family+parents hate me+I hate my parents+I hate school+I hate teacher+I hate dad+I hate mom"]
'''
#years = [i for i in range(2010, 2016)]
#print(years)
#months = [i for i in range(1, 13, 3)] 
#print(months)

### GT Keywords

In [ ]:
mvt_keyword = ["car stolen+find stolen car+report police stolen car+insurance car stolen-dream-check"] 

### Geo Location and Geo Codes

In [ ]:
# Top 50 the most populous cities in the U.S.
region_code_dict = {
    "US-NY-501": "New York NY",
    "US-CA-807": "San Francisco-Oakland-San Jose CA", #including Oakland, CA and San Jose CA
    "US-CA-803": "Los Angeles CA", #including Long Beach, CA
    "US-CA-825": "San Diego CA", 
    "US-CA-862": "Sacramento-Stockton-Modesto CA",
    "US-AZ-753": "Phoenix AZ", #including Mesa, AZ
    "US-MI-505": "Detroit MI",
    "US-NC-517" : "Charlotte NC",
    "US-OH-535" : "Columbus OH",
    "US-FL-561" : "Jacksonville FL",
    "US-MD-511" : "Washington DC (Hagerstown MD)", # including Arlington VA
    "US-OK-650" : "Oklahoma City OK",
    "US-TX-765" : "El Paso TX",
    "US-OR-820" : "Portland OR",
    "US-MD-512" : "Baltimore MD",
    "US-WI-617" : "Milwaukee WI",
    "US-NM-790" : "Albuquerque-Santa Fe NM",
    "US-CA-866" : "Fresno-Visalia CA",
    "US-MO-616" : "Kansas City MO",
    "US-NE-652" : "Omaha NE",
    "US-CO-752" : "Colorado Springs-Pueblo CO",
    "US-NC-560" : "Raleigh-Durham (Fayetteville) NC",
    "US-VA-544" : "Norfolk-Portsmouth-Newport News VA", #including Virginia Beach, VA
    "US-FL-528" : "Miami-Ft. Lauderdale FL",
    "US-CA-800" : "Bakersfield CA",
    "US-OK-671" : "Tulsa OK",
    "US-KS-678" : "Wichita-Hutchinson KS"  ,
    "US-NV-839":"Las Vegas NV",
    "US-WA-819":"Seattle-Tacoma WA",
    "US-CO-751":"Denver CO", #including Aurora, CO
    "US-PA-504":"Philadelphia PA",
    "US-GA-524":"Atlanta GA",
    "US-IN-602":"Chicago IL",
    "US-NH-506":"Boston MA-Manchester NH",
    "CA-ON":"Ontario",
    "US-KY-529" : "Louisville KY",
    "US-TN-640" : "Memphis TN",
    "US-WI-613" : "Minneapolis-St. Paul MN",
    "US-TN-659" : "Nashville TN",
    "US-AZ-789" : "Tucson AZ",
    "US-TX-623" : "Dallas-Ft. Worth TX", # including Fort Worth, TX
    "US-TX-641" : "San Antonio TX",
    "US-TX-618" : "Houston TX",
    "US-TX-635" : "Austin TX",
}






In [ ]:
reversed_dict = {value: key for key, value in region_code_dict.items()}

### Dates (From, To)

In [15]:
From  = ["2015-01-01"]

To = ["2022-12-31"]

## Execute Pytrends to Pull Daily Data from Google Trends

In [17]:

#Get 100 times weekly GT MVT data per city
for times in range(100):
    for geo_code, geo_name in zip(reversed_dict.keys(), reversed_dict.values()):
        df = pd.DataFrame()

        for start_date, end_date in zip(From, To):
            #Get daily estimates with adjusted factors [mean(old/new) * new]
            temp_df = GT_temp_df(start_date, end_date, geo_code, geo_name)
            df = combine_df_and_temp_df_and_rescale_by_overlap_factors(df, temp_df)

        df.name = f"MVT_GT_{geo_name}"
        df.to_csv(f"C:/Users/Sean Liu/Documents/GitHub/GT-MVT-Monthly-City/csv_files/{geo_name}_monthly_{From[0]}_{To[-1]}_{pd.Timestamp.now().strftime('%Y%m%d_%H_%M')}.csv")
        print(f"{geo_name}_monthly_{From[0]}_{To[-1]}_{pd.Timestamp.now().strftime('%Y%m%d_%H_%M')} file saved")

Obtaining GT MVT from 2015-01-01 to 2022-12-31 in New York NY metro area
New York NY_monthly_2015-01-01_2022-12-31_20231016_15_41 file saved
Obtaining GT MVT from 2015-01-01 to 2022-12-31 in San Francisco-Oakland-San Jose CA metro area
San Francisco-Oakland-San Jose CA_monthly_2015-01-01_2022-12-31_20231016_15_51 file saved
Obtaining GT MVT from 2015-01-01 to 2022-12-31 in Los Angeles CA metro area
Los Angeles CA_monthly_2015-01-01_2022-12-31_20231016_16_02 file saved
Obtaining GT MVT from 2015-01-01 to 2022-12-31 in San Diego CA metro area
San Diego CA_monthly_2015-01-01_2022-12-31_20231016_16_13 file saved
Obtaining GT MVT from 2015-01-01 to 2022-12-31 in Sacramento-Stockton-Modesto CA metro area
Sacramento-Stockton-Modesto CA_monthly_2015-01-01_2022-12-31_20231016_16_23 file saved
Obtaining GT MVT from 2015-01-01 to 2022-12-31 in Phoenix AZ metro area
Phoenix AZ_monthly_2015-01-01_2022-12-31_20231016_16_34 file saved
Obtaining GT MVT from 2015-01-01 to 2022-12-31 in Detroit MI metro

RetryError: HTTPSConnectionPool(host='trends.google.com', port=443): Max retries exceeded with url: /trends/api/widgetdata/multiline?req=%7B%22time%22%3A+%222015-01-01+2022-12-31%22%2C+%22resolution%22%3A+%22MONTH%22%2C+%22locale%22%3A+%22en-US%22%2C+%22comparisonItem%22%3A+%5B%7B%22geo%22%3A+%7B%22dma%22%3A+%22535%22%7D%2C+%22complexKeywordsRestriction%22%3A+%7B%22keyword%22%3A+%5B%7B%22type%22%3A+%22BROAD%22%2C+%22value%22%3A+%22car+stolen%22%7D%2C+%7B%22type%22%3A+%22BROAD%22%2C+%22value%22%3A+%22find+stolen+car%22%7D%2C+%7B%22type%22%3A+%22BROAD%22%2C+%22value%22%3A+%22report+police+stolen+car%22%7D%2C+%7B%22type%22%3A+%22BROAD%22%2C+%22value%22%3A+%22insurance+car+stolen-dream-check%22%7D%5D%2C+%22operator%22%3A+%22OR%22%7D%7D%5D%2C+%22requestOptions%22%3A+%7B%22property%22%3A+%22%22%2C+%22backend%22%3A+%22IZG%22%2C+%22category%22%3A+0%7D%2C+%22userConfig%22%3A+%7B%22userType%22%3A+%22USER_TYPE_SCRAPER%22%7D%7D&token=APP6_UEAAAAAZS704BQf1ltpl7nXmZ_IVwlqOVq9mRpM&tz=360 (Caused by R

In [ ]:
test.plot(figsize=(20, 12),  kind ='line')

In [ ]:
[print(i/25*23) for i in test2["car stolen+find stolen car+report police stolen car+insurance car stolen-dream-check"]]

In [ ]:
kw_list=['Facebook', 'Apple', 'Amazon', 'Netflix', 'Google']
pytrend = TrendReq()
#search interest per region
#run model for keywords (can also be competitors)
pytrend.build_payload(kw_list, timeframe='today 1-m')

# Interest by Region
regiondf = pytrend.interest_by_region()
#looking at rows where all values are not equal to 0
regiondf = regiondf[(regiondf != 0).all(1)]

#drop all rows that have null values in all columns
regiondf.dropna(how='all',axis=0, inplace=True)

#visualise
regiondf.plot(figsize=(20, 12), y=kw_list, kind ='bar')

In [ ]:
historicaldf = pytrend.get_historical_interest(['Facebook', 'Apple', 'Amazon', 'Netflix', 'Google'], 
                                               year_start=2020, month_start=1, day_start=1, hour_start=0, 
                                               year_end=2020, month_end=12, day_end=31, hour_end=0, 
                                               cat=0, geo='en-US', gprop='', sleep=0)

#visualise
#plot a timeseries chart
historicaldf.plot(figsize=(20, 12))

#plot seperate graphs, using theprovided keywords
historicaldf.plot(subplots=True, figsize=(20, 12))

In [ ]:
def gt_func(start_date_list, end_date_list, resolution, sample_size, path):
#get monthly GT data
    import inflect
    #give ordinal names to numbers
    p = inflect.engine()

    for t in range(sample_size): #sample from GT 100 times
        #time.sleep(random.randint(21600,21668)) # 6 hrs
        print(f"The {p.ordinal(t+1)} pull from GT")
        qt_list = ["Q1","Q2","Q3","Q4"]
        count = 0
        for i, j in zip(start_date_list, end_date_list):
            df = pd.DataFrame()
            #Q1 - Q4
            if count == 4:
                count = 0
            for k in kw_list:
                #q = the name of the quarter
                q = qt_list[count]
                try:
                    pytrend.build_payload(k, cat=0, timeframe=f'{i.strftime("%Y-%m-%d")} {j.strftime("%Y-%m-%d")}',  geo='US', gprop='')
                    df = pd.concat([df, pytrend.interest_by_region(resolution="DMA", inc_low_vol = False, inc_geo_code = False)], axis = 1)
                    time.sleep(random.randint(120, 130))

                except (Timeout, ResponseError) as e:
                        print(f'!!{i.strftime("%Y-%m-%d")} {j.strftime("%Y-%m-%d")}!! ReadTimeoutError')
                        print(f"Whoops! ReadTimeoutError, the {p.ordinal(t+1)} pull aborted, now we take a rest")
                        print(dt.datetime.now().strftime("%Y/%m/%d_%H:%M:%S"))
                        time.sleep(random.randint(86668, 86888)) # 24 hrs
                        print(f"Rest enough, now we continue!")

                        #continue working
                        pytrend.build_payload(k, cat=0, timeframe=f'{i.strftime("%Y-%m-%d")} {j.strftime("%Y-%m-%d")}',  geo='US', gprop='')
                        df = pd.concat([df, pytrend.interest_by_region(resolution="DMA", inc_low_vol=False, inc_geo_code=False)], axis = 1)
                        time.sleep(random.randint(120, 130))

            df.columns = [f"MVT_{i.strftime('%Y')}_{q}", 
                          f"rape_{i.strftime('%Y')}_{q}", 
                          f"larceny_{i.strftime('%Y')}_{q}", 
                          f"burglary_{i.strftime('%Y')}_{q}", 
                          f"known_deliq_{i.strftime('%Y')}_{q}", 
                          f"low_social_cont_{i.strftime('%Y')}_{q}"]
            df.to_csv(f'{path}gt_crime_{i.strftime("%Y")}_{q}_{dt.datetime.now().strftime("%Y%m%d_%H-%M-%S")}.csv')
            count += 1
        print(f"The {p.ordinal(t+1)} pull from GT Done!!")
        print(f'Time finished: {dt.datetime.now().strftime("%Y/%m/%d, %H:%M")}')
        time.sleep(random.randint(10800,10868)) # 3 hrs


    print(f"Congratulations, Well Done!")